In [1]:
sc

In [2]:
# lectura de fichero
lines = sc.textFile("data/shakespeare.txt")

In [3]:
# uso filter basico
myLines = lines.filter(lambda x: "while" in x)

In [4]:
myLines.first()

u'    But if the while I think on thee (dear friend)'

In [5]:
# contamos las ocurrencias
myLines.count()

315

In [6]:
# creando con parallelize una lista (se puede añadir y eliminar no como en tuplas)
lines2 = sc.parallelize(["pandas", "i like pandas"])

In [7]:
# mostramos todo. Con collect()
lines2.collect()

['pandas', 'i like pandas']

In [10]:
# mostramos algunos datos con take(n)
lines.take(6)

[u'1609', u'', u'THE SONNETS', u'', u'by William Shakespeare', u'']

In [13]:
# otro ejemplo de transformación con union()
myLines2 = lines.filter(lambda line: "because" in line)
totalmyLines = myLines.union(myLines2)

# contamos los valores
totalmyLines.count()

396

In [14]:
# Lo mostramos por líneas
for line in totalmyLines.take(7):
    print line

    But if the while I think on thee (dear friend)
  Now while the world is bent my deeds to cross,
    the less worth. Off with't while 'tis vendible; answer the time
    be conceal'd awhile.
  KING. Wherefore hast thou accus'd him all this while?
    Is as our earing. Fare thee well awhile.
    Our services awhile; but my full heart


# Parte I: Transformations

In [16]:
# Funciones de transformación: map - aplica una función a cada elemento que pasamos.Devuelve un solo elto.
nums = sc.parallelize([1,2,3,4,5])
square = nums.map(lambda x: x**2)
square.collect()
# El collect también puede ir seguido al map.

[1, 4, 9, 16, 25]

In [20]:
# Función map para devolver varios elementos - flatMap. Devuelve 2, 1, 2 cuando coge 1, 1, 1
lines = sc.parallelize(["hello world", "hi", "bye bye"])
words = lines.flatMap(lambda w: w.split(" "))
words.collect()

['hello', 'world', 'hi', 'bye', 'bye']

In [19]:
# Función map para devolver elementos - Map. Devuelve 1, 1, 1, coge 1, 1, 1
lines = sc.parallelize(["hello world", "hi", "bye bye"])
words = lines.map(lambda w: w.split(" "))
words.collect()

[['hello', 'world'], ['hi'], ['bye', 'bye']]

In [29]:
# Otras operaciones a usar: distinct, union, intersection, subtract, cartesian.
elto1 = sc.parallelize(["coffee", "coffee", "panda", "monkey", "tea"])
elto2 = sc.parallelize(["coffee", "monkey", "kitty"])

In [30]:
elto1.distinct().collect()

['coffee', 'tea', 'panda', 'monkey']

In [25]:
elto1.union(elto2).collect()

['coffee', 'coffee', 'panda', 'monkey', 'tea', 'coffee', 'monkey', 'kitty']

In [26]:
elto1.intersection(elto2).collect()

['monkey', 'coffee']

In [27]:
elto1.subtract(elto2).collect()

['tea', 'panda']

In [28]:
elto1.cartesian(elto2).collect()

[('coffee', 'coffee'),
 ('coffee', 'coffee'),
 ('coffee', 'monkey'),
 ('coffee', 'monkey'),
 ('coffee', 'kitty'),
 ('coffee', 'kitty'),
 ('panda', 'coffee'),
 ('monkey', 'coffee'),
 ('tea', 'coffee'),
 ('panda', 'monkey'),
 ('monkey', 'monkey'),
 ('panda', 'kitty'),
 ('monkey', 'kitty'),
 ('tea', 'monkey'),
 ('tea', 'kitty')]

In [39]:
# uso de sample
nums = sc.parallelize([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])
nums.collect()
nums.sample(True, 0.5).collect()

[1, 1, 4, 4, 6, 6, 9, 11, 13]

# Parte II: Actions

In [43]:
# La básica y más conocida: reduce()
mySum = nums.reduce(lambda x,y: x + y)
print(mySum)

120


In [51]:
nums.top(3)

[15, 14, 13]

In [52]:
nums.count()

15

In [53]:
nums.countByValue()

defaultdict(int,
            {1: 1,
             2: 1,
             3: 1,
             4: 1,
             5: 1,
             6: 1,
             7: 1,
             8: 1,
             9: 1,
             10: 1,
             11: 1,
             12: 1,
             13: 1,
             14: 1,
             15: 1})

In [75]:
# Uso de aggregate. Lo que hace es en un valor lleva la suma y en otro un contador.
sumCount = nums.aggregate((0,0), 
                          (lambda acc, value: (acc[0] + value, acc[1] + 1)),
                          (lambda acc1, acc2: (acc1[0] + acc2[0], acc1[1] + acc2[1])))
sumCount[0]/ float(sumCount[1])

8.0

In [54]:
# Otras: take(), takeOrdered, takeSample(), aggregate(), foreach()...

# Parte III Trabajando con Pares Clave/Valor

In [63]:
# Creando manualmente pares clave/valor. creamos pares clave:(primera palabra) , valor:(resto palabras)
pairs = lines.map(lambda x: (x.split(" ")[0],x))
pairs.top(4)

[('hi', 'hi'), ('hello', 'hello world'), ('bye', 'bye bye')]

In [68]:
## Funciones de transformación para poder usar con un RDD par:
# reduceByKey(): combina valores con la misma clave.
# groupByKey(), combineByKey, mapValues, keys(), sortByKey(), values()...

## Funciones de transformación para poder usar con dos RDD par:
# subtractByKey(other), join(other), rightOuterJoin(other), leftOuterJoin(other), cogroup(other)...

# Ejemplo de uso: 
result = pairs.filter(lambda x: len(x[1]) > 2)

In [69]:
result.collect()

[('hello', 'hello world'), ('bye', 'bye bye')]

In [76]:
# Uso de map y reduceByKey. Cuando hacemos reduceByKey, el campo clave no cuenta, por eso suma solo x.
# Tambien se puede usar con combineByKey.
lines = sc.textFile("data/shakespeare.txt")
words = lines.flatMap(lambda x: x.split(" "))
result = words.map(lambda x: (x, 1)).reduceByKey(lambda x,y: x + 1)

In [80]:
result.take(5)

[(u'', 249696),
 (u'fawn', 7),
 (u'Fame,', 2),
 (u'mustachio', 1),
 (u'protested,', 1)]

In [82]:
# Uso también de sortByKey(), countByKey()
result.sortByKey().take(5)

[(u'', 249696), (u'"', 181), (u'"\'Tis', 1), (u'"A', 4), (u'"Air,"', 1)]

In [84]:
# Devuelve todos los valores asociados a una clave dada.
result.lookup('fawn')

[7]

# Parte IV Lectura y grabación de datos

In [ ]:
input = sc.textFile("file:///home/")